In [1]:
# https://huggingface.co/blog/sentiment-analysis-python

In [2]:
def news_df_daily():
    start_date = date(2021, 1, 1)
    end_date = date(2023, 8, 30)  
    news = []

    title_list = []
    published_date = []
    article_link = []
    article_text = []
    start_time = time.time()  # Record the start time of the iteration
    while start_date <= end_date:  # Changed the condition to include the end date


        # Set the date and period
        gnews = GNews(language='en', country='US',
                      max_results=5,exclude_websites=['www.seattletimes.com', 'www.fiercepharma.com', 
                        'www.governor.virginia.gov','www.genengnews.com','www.thelancet.com',
                        'www.genomeweb.com',' www.fiercebiotech.com',' www.fiercebiotech.com',
                        'www.prnewswire.com','onlinelibrary.wiley.com'])

        gnews.start_date = (start_date.year, start_date.month, start_date.day)
        start_date += timedelta(days=1)
        gnews.end_date = (start_date.year, start_date.month, start_date.day)

        # Get the news results
        results = gnews.get_news('Thermo Fisher')

        for i in results:
            try:
                article = gnews.get_full_article(i['url'])
                print("--", article)
            except:
                article = gnews.get_full_article("Error")
            try:
                article_link.append(i['url'])
            except:
                article_link.append("Error")
            try:
                title_list.append(i['title'])
            except:
                title_list.append("Error")
            try:
                article_text.append(article.text)
            except:
                article_text.append("Error")
            try:
                parsed_date = datetime.strptime(i['published date'], "%a, %d %b %Y %H:%M:%S GMT")
                published_date.append(parsed_date.strftime("%Y-%m-%d"))
            except:
                published_date.append("Error")

        news_data = {
            'Title': title_list,
            'Published Date': published_date,
            'URL': article_link,
            'Article Text': article_text
        }

        df = pd.DataFrame(news_data)

        time.sleep(10)
    end_time = time.time()  # Record the end time of the iteration
    iteration_time = end_time - start_time
    print(f"Iteration time: {iteration_time} seconds")
    return df

In [3]:
def percentage(part,whole):
    return 100 * float(part)/float(whole)

In [4]:
def word_cloud(text):
    # Create a set for custom stopwords from "stopwords_extend.txt"
    custom_stopwords = set()
    with open("stopwords_extend.txt", "r") as file:
        custom_stopwords.update(line.strip() for line in file)

    # Combine stopwords from all sources into a single set
    all_stopwords = (
        set(STOPWORDS) | set(stopwords_gensim) | stopwords_nltk | custom_stopwords
    )

    # Print or use the extended stopwords set as needed
    # print("Extended Stopwords Set:", all_stopwords)

    stopwords = set(all_stopwords)
    additional_stopwords=["Error","Thermo Fisher"]
    if additional_stopwords is not None:
        stopwords.update(additional_stopwords)
    allWords = ' '.join([nws for nws in text])
    wordCloud = WordCloud(background_color='black',width = 1600, height = 800,stopwords = stopwords,min_font_size = 20,max_font_size=150,colormap='prism').generate(allWords)
    fig, ax = plt.subplots(figsize=(20,10), facecolor='k')
    plt.imshow(wordCloud)
    ax.axis("off")
    fig.tight_layout(pad=0)
    plt.show()

In [5]:
def clean_text(text):
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # remove urls
    text = re.sub(r'\@\w+|\#', '', text)  # remove at and hash tags
    text = text.encode('ascii', 'ignore').decode('ascii')  # remove emojis
    text = re.sub(r'\s+', ' ', text).strip()  # remove extra spaces
    return text

In [6]:
def piechart_viz(thermo_news_df_quarter):
    #Assigning Initial Values
    positive = 0
    negative = 0
    neutral = 0
    #Creating empty lists
    news_list = []
    neutral_list = []
    negative_list = []
    positive_list = []
    sentiment = []

    #Iterating over the tweets in the dataframe
    for news in thermo_news_df_quarter['Article Text']:
        
#         if news!="Error":
        news_list.append(news)
        analyzer = SentimentIntensityAnalyzer().polarity_scores(news)
        neg = analyzer['neg']
        neu = analyzer['neu']
        pos = analyzer['pos']
        comp = analyzer['compound']

        if neg > pos:
            negative_list.append(news) #appending the news that satisfies this condition
            sentiment.append("Negative")
            negative += 1 #increasing the count by 1
        elif pos > neg:
            positive_list.append(news) #appending the news that satisfies this condition
            sentiment.append("Positive")
            positive += 1 #increasing the count by 1
        elif pos == neg:
            neutral_list.append(news) #appending the news that satisfies this condition
            sentiment.append("Neutral")
            neutral += 1 #increasing the count by 1

    thermo_news_df_quarter['sentiment']=sentiment
    
    positive = percentage(positive, len(sentiment)) #percentage is the function defined above
    negative = percentage(negative, len(sentiment))
    neutral = percentage(neutral, len(sentiment))

#     news_list = pd.DataFrame(news_list)
#     neutral_list = pd.DataFrame(neutral_list)
#     negative_list = pd.DataFrame(negative_list)
#     positive_list = pd.DataFrame(positive_list)
    print("Positive Sentiment:", '%.2f' % len(positive_list), end='\n')
    print("Neutral Sentiment:", '%.2f' % len(neutral_list), end='\n')
    print("Negative Sentiment:", '%.2f' % len(negative_list), end='\n')

    #Creating PieCart
    labels = ['Positive ['+str(round(positive))+'%]' , 'Neutral ['+str(round(neutral))+'%]','Negative ['+str(round(negative))+'%]']
    sizes = [positive, neutral, negative]
    colors = ['yellowgreen', 'blue','red']
    patches, texts = plt.pie(sizes,colors=colors, startangle=90)
    plt.style.use('default')
    plt.legend(labels)
    plt.title("Sentiment Analysis Result for Thermofisher" )
    plt.axis('equal')
    plt.show()
    
    return thermo_news_df_quarter

In [7]:
from datetime import datetime

import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

from gnews import GNews
from datetime import datetime
from textsum.summarize import Summarizer
from tqdm import tqdm
import time

from datetime import date, timedelta
import time
import re

from wordcloud import WordCloud, STOPWORDS
from gensim.parsing.preprocessing import STOPWORDS as stopwords_gensim
from nltk.corpus import stopwords
stopwords_nltk = set(stopwords.words('english'))

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import warnings
warnings.filterwarnings('ignore')

11/14/2023 08:30:09 PM - Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [ ]:
thermo_news_df_daily=news_df_daily()

-- <newspaper.article.Article object at 0x0000022FBEF11B70>
-- <newspaper.article.Article object at 0x0000022FCD37E260>
-- <newspaper.article.Article object at 0x0000022FCD37C490>
-- <newspaper.article.Article object at 0x0000022FCD37C6A0>
-- <newspaper.article.Article object at 0x0000022FCD37E260>
-- <newspaper.article.Article object at 0x0000022FE91AFE20>
-- <newspaper.article.Article object at 0x0000022FCD37C6A0>
-- <newspaper.article.Article object at 0x0000022FE914E830>
-- <newspaper.article.Article object at 0x0000022FCD37E260>
-- <newspaper.article.Article object at 0x0000022FE914DFC0>
-- <newspaper.article.Article object at 0x0000022FCD37D570>
-- <newspaper.article.Article object at 0x0000022FE8E6B9D0>
-- <newspaper.article.Article object at 0x0000022FE914F670>
-- <newspaper.article.Article object at 0x0000022FE914DFC0>
-- <newspaper.article.Article object at 0x0000022FE91F3D60>
-- <newspaper.article.Article object at 0x0000022FE8E6B9D0>
-- <newspaper.article.Article object at 

11/14/2023 08:33:01 PM - Article `download()` failed with 403 Client Error: Forbidden for url: https://www.pnas.org/doi/10.1073/pnas.2015149118 on URL https://news.google.com/rss/articles/CBMiMGh0dHBzOi8vd3d3LnBuYXMub3JnL2RvaS8xMC4xMDczL3BuYXMuMjAxNTE0OTExONIBAA?oc=5&hl=en-US&gl=US&ceid=US:en


-- None
-- <newspaper.article.Article object at 0x0000022FCD37C9D0>
-- <newspaper.article.Article object at 0x0000022FE91AFE20>
-- <newspaper.article.Article object at 0x0000022FCD37E110>
-- <newspaper.article.Article object at 0x0000022FE8E6B820>
-- <newspaper.article.Article object at 0x0000022FCD37ECB0>
-- <newspaper.article.Article object at 0x0000022FCD37E110>
-- <newspaper.article.Article object at 0x0000022FE91AFE20>
-- <newspaper.article.Article object at 0x0000022FE8E6B820>
-- <newspaper.article.Article object at 0x0000022FCD37ECB0>
-- <newspaper.article.Article object at 0x0000022FE91AFEB0>
-- <newspaper.article.Article object at 0x0000022FE91AFE20>
-- <newspaper.article.Article object at 0x0000022FE8E6B9D0>
-- <newspaper.article.Article object at 0x0000022FE8E6AEC0>
-- <newspaper.article.Article object at 0x0000022FE914EBF0>
-- <newspaper.article.Article object at 0x0000022FE8E6B8B0>
-- <newspaper.article.Article object at 0x0000022FE91F3DC0>
-- <newspaper.article.Article ob

11/14/2023 08:35:18 PM - Article `download()` failed with 403 Client Error: Max restarts limit reached for url: https://www.forbes.com/sites/brucejapsen/2021/01/19/thermo-fisher-to-buy-covid-19-test-maker-mesa-biotech-for-450-million/ on URL https://news.google.com/rss/articles/CBMiemh0dHBzOi8vd3d3LmZvcmJlcy5jb20vc2l0ZXMvYnJ1Y2VqYXBzZW4vMjAyMS8wMS8xOS90aGVybW8tZmlzaGVyLXRvLWJ1eS1jb3ZpZC0xOS10ZXN0LW1ha2VyLW1lc2EtYmlvdGVjaC1mb3ItNDUwLW1pbGxpb24v0gF-aHR0cHM6Ly93d3cuZm9yYmVzLmNvbS9zaXRlcy9icnVjZWphcHNlbi8yMDIxLzAxLzE5L3RoZXJtby1maXNoZXItdG8tYnV5LWNvdmlkLTE5LXRlc3QtbWFrZXItbWVzYS1iaW90ZWNoLWZvci00NTAtbWlsbGlvbi9hbXAv?oc=5&hl=en-US&gl=US&ceid=US:en


-- <newspaper.article.Article object at 0x0000022FE91AFF10>
-- None
-- <newspaper.article.Article object at 0x0000022FE91AFEB0>
-- <newspaper.article.Article object at 0x0000022FE91AFF10>


...

-- None


11/15/2023 09:59:12 AM - 2


-- None


11/15/2023 09:59:13 AM - 2


-- None


In [ ]:
thermo_news_df_daily

In [ ]:
thermo_news_df_daily=thermo_news_df_daily.reset_index(drop=True)
thermo_news_df_daily['Published Date']=pd.to_datetime(thermo_news_df_daily['Published Date'])
thermo_news_df_daily['day']=thermo_news_df_daily['Published Date'].dt.day
thermo_news_df_daily['quarter']=thermo_news_df_daily['Published Date'].dt.quarter
thermo_news_df_daily['month']=thermo_news_df_daily['Published Date'].dt.month
thermo_news_df_daily['year']=thermo_news_df_daily['Published Date'].dt.year
thermo_news_df_daily=thermo_news_df_daily.sort_values(['Published Date'],ascending=True)
thermo_news_df_daily.head()

In [ ]:
thermo_news_df_daily['Article Text Cleaned'] = thermo_news_df_daily['Article Text'].astype(str).apply(clean_text)  


In [ ]:
thermo_news_df_daily['year'].unique()

In [ ]:
thermo_news_df_daily['month'].unique()

In [ ]:
thermo_news_df_daily['day'].unique()

In [ ]:
# thermo_news_df_monthly_select_sentiment = pd.concat([thermo_news_df_monthly_select_sentiment, thermo_news_df_monthly_select])


In [ ]:
import pandas as pd

# Initialize an empty DataFrame to store the results
thermo_news_df_daily_select_sentiment_all=pd.DataFrame()

for year_i in thermo_news_df_daily['year'].unique():
    for month_i in thermo_news_df_daily['month'].unique():
        if (year_i == 2023 and month_i > 8):
            print("Skipping Year:", year_i, "Month:", month_i)
            continue  # Skip this iteration and move to the next year/month

        print("Processing Year:", year_i, "Month:", month_i)

        # Filter the DataFrame based on year and month
        thermo_news_df_daily_select = thermo_news_df_daily[
            (thermo_news_df_daily['month'] == month_i) &
            (thermo_news_df_daily['year'] == year_i) & (thermo_news_df_daily['Article Text'] != "Error") & \
            (thermo_news_df_daily['Article Text'] != "")
        ]

        # Assuming piechart_viz returns a DataFrame with sentiment
        
        try:
            thermo_news_df_daily_select_sentiment = piechart_viz(thermo_news_df_daily_select)
            thermo_news_df_daily_select_sentiment_all =pd.concat([thermo_news_df_daily_select_sentiment_all,thermo_news_df_daily_select_sentiment])
            display(thermo_news_df_daily_select_sentiment)
            
        except:
            print("No Pie Chart")
        try:
            print("Wordcloud: Positive for Thermofisher for Year "+str(year_i)+" Month: "+str(month_i))
            word_cloud(thermo_news_df_daily_select_sentiment[thermo_news_df_daily_select_sentiment['sentiment']=="Positive"]['Article Text Cleaned'].values)
        except:
            print("No wordcloud")
        try:
            print("Wordcloud: Negative for Thermofisher for Year "+str(year_i)+" Month: "+str(month_i))
            word_cloud(thermo_news_df_daily_select_sentiment[thermo_news_df_daily_select_sentiment['sentiment']=="Negative"]['Article Text Cleaned'].values)
        except:
            print("No wordcloud")
        


In [ ]:
thermo_news_df_daily_select_sentiment_all

In [30]:
grouped = thermo_news_df_daily_select_sentiment_all.groupby(['day','month', 'year', 'sentiment']).size().reset_index(name='count')
sentiments = pd.pivot_table(grouped, values='count', index=['day','month', 'year'], columns='sentiment', fill_value=0).reset_index()
sentiments.columns = ['sentiment'+'_'+(col) for col in sentiments.columns]
sentiments.head()

,sentiment_day,sentiment_month,sentiment_year,sentiment_Negative,sentiment_Neutral,sentiment_Positive
0,1,1,2021,0,0,1
1,1,1,2022,0,0,2
2,1,2,2021,0,0,5
3,1,2,2022,0,0,4
4,1,3,2021,0,0,4


In [31]:
grouped

,day,month,year,sentiment,count
0,1,1,2021,Positive,1
1,1,1,2022,Positive,2
2,1,2,2021,Positive,5
3,1,2,2022,Positive,4
4,1,3,2021,Positive,4
...,...,...,...,...,...
838,31,8,2022,Positive,2
839,31,10,2021,Negative,1
840,31,10,2021,Positive,3
841,31,12,2021,Negative,1


In [32]:
sentiments['sentiment_Negative'].max()

4

In [34]:
sentiments['sentiment_Positive'].max()

8

In [35]:
sentiments.to_excel("news_sentiment.xlsx",index=False)